In [1]:
import pandas as pd
import re
import json
import requests
from xml.etree import ElementTree
import time
from dotenv import load_dotenv
import ast  # To safely evaluate string representation of a list
import bdikit as bdi
import requests
import os

# Extract Ground Truth

This Notebook takes as input the Export File from ProteomeCentral: https://proteomecentral.proteomexchange.org/: proteomexchange_search.tsv


The final output will be a json file with ground truth public information on datasets

In [2]:
export_df = pd.read_csv('exp_input/proteomexchange_search.tsv', sep='\t')
print(f"number of dataset ids in export dataframe from ProteomeCentral: {len(export_df)}")
datasets_df = export_df[['publication','identifier','repository','title','keywords']]

number of dataset ids in export dataframe from ProteomeCentral: 40137


## publications values frequencies

to get an idea of the most frequent values in df publication, for all the dataset ids from the source file

In [3]:
export_df['publication'].value_counts()

publication
Dataset with its publication pending                                                                                                                                                                      11719
no publication                                                                                                                                                                                             2550
<a href="http://www.ncbi.nlm.nih.gov/pubmed/35084980" target="_blank">Melani et al. (2022)</a>                                                                                                               56
<a href="http://www.ncbi.nlm.nih.gov/pubmed/28267743" target="_blank">Matsumoto et al. (2017)</a>                                                                                                            28
<a href="http://www.ncbi.nlm.nih.gov/pubmed/28071820" target="_blank">Kreutz et al. (2017)</a>                                                              

Clean the `publication` column by filtering out unwanted values like `Dataset with its publication pending`, `no publication`. Remove rows with unwanted values

In [4]:
filtered_df = export_df[~export_df['publication'].isin(["Dataset with its publication pending", "no publication"])]
print(len(filtered_df),'\n',filtered_df.columns)

25868 
 Index(['identifier', 'title', 'repository', 'species', 'instrument',
       'publication', 'lab_head', 'announce_date', 'keywords'],
      dtype='object')


## PMID and DOI Mappings extraction functions for future use

In [5]:
def PMID_to_doi(pmid):
    if pmid in pmid_doi_mapping:
        return pmid_doi_mapping[pmid]
    
    else:
        print(f"PMID {pmid} not found in mapping file. Querying API...")
    
        base = "https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/"
        params = {"tool": "mytool", "email": "myemail@example.com", "ids": pmid, "format": "json"}    
        
        response = requests.get(base, params=params)
        
        if response.status_code == 200:
            data = response.json()
            records = data.get("records", [])
            if records and "doi" in records[0]:
                print(f"PMID: {pmid}, DOI: {records[0]['doi']}")
                pmid_doi_mapping[pmid] = records[0]["doi"]  # Store in mapping
                return records[0]["doi"]
            else:
                print(f"PMID: {pmid}, DOI: xxxx")  # No doi found
                return None  # No doi found
        else:
            print(f"API request failed for PMID {pmid}")
            return None  # Request failed

def url_to_doi(url):
    if "dx.doi.org" in url:
        match = re.search(r'dx.doi.org/([a-zA-Z0-9./-]+)', url)
        if match:
            return match.group(1)
        else:
            print(f"DOI not found in {url}")
            return None
    elif "pubmed" in url:
        match = re.search(r'pubmed/([0-9]+)', url)
        if match:
            return PMID_to_doi(match.group(1))
        else:
            print(f"PMID not found in {url}")
            return None        

def PMID_to_url(pmid):
    base_url = "https://www.ncbi.nlm.nih.gov/pubmed/"
    return base_url + str(pmid)

def url_to_pmid(url):
    if "pubmed" in url:
        match = re.search(r'pubmed/([0-9]+)', url)
        if match:
            return match.group(1)
        else:
            print(f"PMID not found in {url}")
            return None
    else:
        print(f"URL does not contain a PMID: {url}") if 'doi' not in url else None
        return None
    
def batch_PMID_to_doi(pmids, batch_size=100):
    base_url = "https://www.ncbi.nlm.nih.gov/pmc/utils/idconv/v1.0/"
    results = {}

    for i in range(0, len(pmids), batch_size):
        progress = i / len(pmids) * 100
        print(f"Processing batch {i}-{i+batch_size} ({progress:.2f}%)")
        batch = pmids[i:i+batch_size]  # Get a batch of PMIDs
        params = {"tool": "mytool", "email": "myemail@example.com", "ids": ",".join(batch), "format": "json"}
        
        response = requests.get(base_url, params=params)
        
        if response.status_code == 200:
            data = response.json()
            records = data.get("records", [])

            for record in records:
                pmid = record.get("pmid")
                doi = record.get("doi", None)  # Get DOI if available
                
                if pmid and pmid not in results:
                    results[pmid] = doi  # Store in dictionary
        
        else:
            print(f"API request failed for batch {i}-{i+batch_size}: {response.status_code}")
        
        time.sleep(0.1)  # Prevent hitting API rate limits (adjust as needed)

    return results

In [6]:
# Load Mappings
pmid_doi_mapping_file = "exp_output/pmid_doi_mapping.json"
if os.path.exists("exp_output/pmid_doi_mapping.json"):
    
    with open(pmid_doi_mapping_file, "r") as f:
        pmid_doi_mapping = json.load(f)  # read json file
        print(f"Loaded {len(pmid_doi_mapping)} mappings from file.")
else:
    pmid_doi_mapping = {}

Loaded 144993 mappings from file.


## extract the citing Publications for each dataset

Create a new column for the links of the citing publications for each dataset

In [7]:
filtered_df = filtered_df.copy()

filtered_df.loc[:, 'citing_publications_links'] = None

for i, row in filtered_df.iterrows():
    pub = str(row['publication'])  # Ensure string type
    if "href" in pub:
        match = re.findall(r'href=[\'"]([^\'"]+)[\'"]', pub)  # Extract href links
        filtered_df.at[i, 'citing_publications_links'] = match if match else None
    else:
        filtered_df.at[i, 'citing_publications_links'] = None

# Drop rows with missing links safely
filtered_df = filtered_df.dropna(subset=['citing_publications_links']).reset_index(drop=True)

print(f"# n datasets with publications: {len(filtered_df)}") 
print(filtered_df.columns)

# n datasets with publications: 25696
Index(['identifier', 'title', 'repository', 'species', 'instrument',
       'publication', 'lab_head', 'announce_date', 'keywords',
       'citing_publications_links'],
      dtype='object')


In [8]:
m = 0
for i,row in filtered_df.iterrows():
    id = row['identifier']
    m+=len(row['citing_publications_links'])
print(f"Total number of publications: {m}")
print(f"Average number of publications per dataset: {m/len(filtered_df)}")

Total number of publications: 39347
Average number of publications per dataset: 1.53125


## Update Mapping

In [9]:
urls = filtered_df['citing_publications_links']
print(len(urls))
urls = urls.explode()
print(len(urls))
urls = urls.dropna()
print(len(urls))
pmids = urls.apply(url_to_pmid).dropna()
print(len(set(pmids)))
results = batch_PMID_to_doi(list(set(pmids)),150)
print(len(results))

25696
39347
39347
PMID not found in https://www.ncbi.nlm.nih.gov/pubmed/PMC10340108
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/IPX0003899000
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/IPX0003216000
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/TBD
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/
19652
Processing batch 0-150 (0.00%)
Processing batch 150-300 (0.76%)
Processing batch 300-450 (1.53%)
Processing batch 450-600 (2.29%)
Processing batch 600-750 (3.05%)
Processing batch 750-900 (3.82%)
Processing batch 900-1050 (4.58%)
Processing batch 1050-1200 (5.34%)
Processing batch 1200-1350 (6.11%)
Processing batch 1350-1500 (6.87%)
Processing batch 1500-1650 (7.63%)
Processing batch 1650-1800 (8.40%)
Processing batch 1800-1950 (9.16%)
Processing batch 1950-2100 (9.92%)
Processing batch 2100-2250 (10.69%)
Processing batch 2250-2400 (11.45%)
Processing

In [10]:
for key, value in results.items():
    if key not in pmid_doi_mapping:
        pmid_doi_mapping[key] = value

print(len(pmid_doi_mapping))

144993


In [11]:
# save to file
with open(pmid_doi_mapping_file, "w") as f:
    json.dump(pmid_doi_mapping, f, indent=4) # Save to JSON
print(f"Saved {len(pmid_doi_mapping)} mappings to JSON file.")

Saved 144993 mappings to JSON file.


In [12]:
filtered_df = filtered_df.explode('citing_publications_links')  # Split lists into rows

df_grouped = filtered_df.groupby('citing_publications_links').apply(lambda x: {
    "citing_publication_link": x.name,  # Grouped key
    "citing_pmid": url_to_pmid(x.name), 
    "citing_doi": url_to_doi(x.name),  # Extract DOI from URL
    "cited_datasets_count": len(x),
    "cited_datasets": x.apply(lambda row: {
        "dataset_id": row["identifier"] if pd.notna(row["identifier"]) else "n/a",
        "dataset_repo": row["repository"] if pd.notna(row["repository"]) else "n/a",
        "dataset_keywords": row["keywords"] if pd.notna(row["keywords"]) else "n/a",
        "dataset_title": row["title"] if pd.notna(row["title"]) else "n/a",
    }, axis=1).tolist()
}).tolist()

json_output = json.dumps(df_grouped, indent=4)

with open("exp_ground_truth/publications_ground_truth.json", "w") as json_file:
    json_file.write(json_output)

with open(pmid_doi_mapping_file, "w") as f:
    json.dump(pmid_doi_mapping, f, indent=4) # Save to JSON
    
print(f"Saved {len(df_grouped)} entries to JSON file.")
print(f"Saved {len(pmid_doi_mapping)} mappings to JSON file.")

PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/
PMID 0 not found in mapping file. Querying API...
PMID: 0, DOI: xxxx
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/IPX0003216000
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/IPX0003216000
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/IPX0003899000
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/IPX0003899000
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/TBD
PMID not found in http://www.ncbi.nlm.nih.gov/pubmed/TBD
PMID 0 not found in mapping file. Querying API...
PMID: 0, DOI: xxxx
PMID not found in https://www.ncbi.nlm.nih.gov/pubmed/PMC10340108
PMID not found in https://www.ncbi.nlm.nih.gov/pubmed/PMC10340108
Saved 34879 entries to JSON file.
Saved 144993 mappings to JSON file.


## Integration of dataset references from Gene Expression Omnibus   

We want to integrate the GEO datasets

In [13]:
input_ground_truth_file = "exp_ground_truth/publications_ground_truth.json"
output_ground_truth_file = "exp_ground_truth/publications_ground_truth_PXD_GSE.json"
GEO_extract = "exp_input/GEO_data.csv"

### Extracting all the datasets with eutils api

We get all the uids of GEO series datasets

In [14]:
esearch_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params = {
    "db": "gds",  # Search in GEO database
    "term": '"gse"[Entry Type]',  # Only fetch GEO Series (GSE)
    "retmax": "1000000",  # Maximum records to retrieve (adjust as needed)
    "retmode": "xml"
}

root = ElementTree.fromstring(requests.get(esearch_url, params=params).content)

gse_ids = [id_elem.text for id_elem in root.findall(".//Id")]

print(f"Total datasets found: {len(gse_ids)}")
print("Sample GSE IDs:", gse_ids[:10])  # Show first 10 results

Total datasets found: 248808
Sample GSE IDs: ['200291629', '200291614', '200291440', '200291300', '200291298', '200291297', '200291295', '200291292', '200291265', '200291226']


In [15]:
def fetch_GEO_data(IDs,request_url,start,stop):
    params = {
        "db": "gds",
        "id": ",".join(IDs[start:stop]),  # Query window
        "retmode": "json"
    }
    response = requests.get(request_url, params=params)
    
    try:
        data = response.json()
    except:
        raise ValueError("Failed to parse JSON response! Please check the response content.")
    
    return data

In [16]:
fetch_GEO_data(gse_ids,"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi",0,10)

{'header': {'type': 'esummary', 'version': '0.3'},
 'result': {'uids': ['200291629',
   '200291614',
   '200291440',
   '200291300',
   '200291298',
   '200291297',
   '200291295',
   '200291292',
   '200291265',
   '200291226'],
  '200291629': {'uid': '200291629',
   'accession': 'GSE291629',
   'gds': '',
   'title': 'The Role of BAZ2-dependent Chromatin Remodeling in Suppressing G4 DNA Structures and Associated Genomic Instability [INDUCE-seq]',
   'summary': 'DNA G-quadruplexes (G4s) are secondary structures with significant roles in regulating genome function and stability. Dysregulation of the dynamic formation of G4s is linked to genomic instability and disease, but the underlying mechanisms are not fully understood. In this study, we conducted a screen of chromatin-modifying enzymes and identified nine potential inhibitors of G4 formation, including seven that were not previously characterized. Among these, we highlight the role of BAZ2 chromatin remodelers as key suppressors o

In [17]:
GEO_series_data = {}
i = 0
mxm = 300
while True:
    print(f"Progress: {i / len(gse_ids) * 100} %")
    if i > len(gse_ids):
        break
    try:
        new_data = fetch_GEO_data(gse_ids,"https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esummary.fcgi",i,mxm)
        #print(f"Data: {new_data}")
        for uid, details in new_data["result"].items():
            if uid == "uids":  # Ignore metadata key
                continue
            GEO_series_data[uid] = details
    except:
        print(f"Error at {i}, {mxm}")
            
    i += 300
    mxm += 300
    time.sleep(0.005) 

Progress: 0.0 %
Progress: 0.12057490112858107 %
Progress: 0.24114980225716215 %
Progress: 0.3617247033857432 %
Progress: 0.4822996045143243 %
Progress: 0.6028745056429053 %
Progress: 0.7234494067714864 %
Progress: 0.8440243079000675 %
Progress: 0.9645992090286486 %
Progress: 1.0851741101572296 %
Progress: 1.2057490112858107 %
Progress: 1.326323912414392 %
Progress: 1.4468988135429728 %
Progress: 1.567473714671554 %
Progress: 1.688048615800135 %
Progress: 1.808623516928716 %
Progress: 1.9291984180572972 %
Progress: 2.0497733191858782 %
Progress: 2.1703482203144593 %
Progress: 2.2909231214430403 %
Progress: 2.4114980225716214 %
Progress: 2.5320729237002024 %
Progress: 2.652647824828784 %
Progress: 2.7732227259573645 %
Progress: 2.8937976270859456 %
Progress: 3.014372528214527 %
Progress: 3.134947429343108 %
Progress: 3.255522330471689 %
Progress: 3.37609723160027 %
Progress: 3.4966721327288517 %
Progress: 3.617247033857432 %
Progress: 3.7378219349860133 %
Progress: 3.8583968361145944 %
P

In [18]:
pmids_to_gse_mapping = {} # mapping of pubmed ids to accession codes of GEO datasets
not_matched_GSEs = []
no_pmid = 0
pmids_found = []
cnt=0

for entry in GEO_series_data: # iterate dataset entries
    if type(GEO_series_data[entry]['pubmedids']) == list and len(GEO_series_data[entry]['pubmedids']) == 0:
        not_matched_GSEs.append(GEO_series_data[entry]['accession'])
        
    elif type(GEO_series_data[entry]['pubmedids']) == list:
        pmids_found.extend(GEO_series_data[entry]['pubmedids'])
        cnt += 1
        for pmid in GEO_series_data[entry]['pubmedids']:
            if pmid not in pmids_to_gse_mapping:
                pmids_to_gse_mapping[pmid] = [GEO_series_data[entry]['accession']]
            else:
                pmids_to_gse_mapping[pmid].append(GEO_series_data[entry]['accession'])  # Append separately

    else:
        print(f"Check why pmid not extracted at: {entry}!")

print(f"Unique PMIDs found: {len(pmids_to_gse_mapping)}")
print(f"# of Datasets with a pmid citation: {cnt}")
print(f"# of Datasets without a pmid citation: {len(not_matched_GSEs)}")

Unique PMIDs found: 127307
# of Datasets with a pmid citation: 194205
# of Datasets without a pmid citation: 54303


In [20]:
pmid_doi_mapping_file = "exp_output/pmid_doi_mapping.json"
if os.path.exists("exp_output/pmid_doi_mapping.json"):
    
    with open(pmid_doi_mapping_file, "r") as f:
        pmid_doi_mapping = json.load(f)  # read json file
else:
    pmid_doi_mapping = batch_PMID_to_doi(list(set(pmids_found)))
    
    os.makedirs(os.path.dirname(pmid_doi_mapping_file), exist_ok=True)
    with open(pmid_doi_mapping_file, "w") as f:
        json.dump(pmid_doi_mapping, f, indent=4) # Save to JSON
        
# reverse the mapping
doi_to_pmid_mapping = {}

for pmid, doi in pmid_doi_mapping.items():
    if isinstance(doi, list):  # If DOI is a list (multiple DOIs per PMID)
        for d in doi:
            doi_to_pmid_mapping.setdefault(d, []).append(pmid)  # Store multiple PMIDs in a list
    else:
        doi_to_pmid_mapping.setdefault(doi, []).append(pmid)  # Always store as list

# If needed, convert single-item lists to just the value
doi_to_pmid_mapping = {k: v[0] if len(v) == 1 else v for k, v in doi_to_pmid_mapping.items()}

In [21]:
# count how many times dx.doi.org is in df_groud_truth['publication']
with open(input_ground_truth_file, "r") as f:
    input_ground_truth = json.load(f)

count, pmcnt = 0,0

for entry in input_ground_truth: # Iterate through the JSON data
    publication = entry.get("citing_publication_link", "")  # Safely get citing_publication_link field

    if isinstance(publication, str):  # Ensure publication is a string
        if "dx.doi.org" in publication:
            count += 1
        elif "pubmed" in publication:
            pmcnt += 1

print(f"Total dx.doi.org found: {count}")
print(f"Total pubmed found: {pmcnt}")
print(f"Total publications: {len(input_ground_truth)}")
print(f"Total matches: {count + pmcnt}")

Total dx.doi.org found: 14887
Total pubmed found: 19992
Total publications: 34879
Total matches: 34879


In [22]:
# for each entry in ground truth, we will create a column with GSE ids. We can use the publication_col to find the DOI or the PM id
# then we can use the mapping {pmid -> doi} to find the PM id for doi
# then we can use the mapping {pmid -> gse} to find the GSE id for the PM id
import re

# Convert `data` into a lookup dictionary using `accession` as the key
gse_lookup = {v["accession"]: v for v in GEO_series_data.values()}

count, pmcnt = 0, 0
cnt_tot_pm, cnt_tot_doi = 0, 0

skip_PXD_dataset_doi = 0

for entry in input_ground_truth:

    publication = entry.get("citing_publication_link", "")
    
    # skip dois of PXD datasets
    if publication and "//dx.doi.org/10.6019/PXD" in publication:
        skip_PXD_dataset_doi += 1
        continue

    if publication and "dx.doi.org" in publication:
        match = re.search(r'dx.doi.org/([a-zA-Z0-9./-]+)', publication)
        if match:
            doi = match.group(1)
            count += 1
            pmid = doi_to_pmid_mapping.get(doi)
            if pmid:
                gse_ids = pmids_to_gse_mapping.get(pmid, [])  # Ensure it's a list

                for gse_id in gse_ids:  # Iterate over multiple GSEs
                    cnt_tot_doi += 1
                    dataset_info = gse_lookup.get(gse_id, {})  # Get dataset info

                    entry.setdefault("cited_datasets", []).append({
                        "dataset_id": gse_id,
                        "dataset_repo": "GEO",
                        "dataset_summary": dataset_info.get("summary", "N/A"),
                        "dataset_title": dataset_info.get("title", "N/A")
                    })

    elif publication and "pubmed" in publication:
        match = re.search(r'pubmed/([0-9]+)', publication)
        if match:
            pmid = match.group(1)
            pmcnt += 1
            gse_ids = pmids_to_gse_mapping.get(pmid, [])  # Ensure it's a list

            for gse_id in gse_ids:  # Iterate over multiple GSEs
                cnt_tot_pm += 1
                dataset_info = gse_lookup.get(gse_id, {})

                entry.setdefault("cited_datasets", []).append({
                    "dataset_id": gse_id,
                    "dataset_repo": "GEO",
                    "dataset_summary": dataset_info.get("summary", "N/A"),
                    "dataset_title": dataset_info.get("title", "N/A")
                })

# Print results
print(f"Total dx.doi.org found: {count}")
print(f"Total pubmed found: {pmcnt}")
print(f"Total matches of gse accession codes: in DOI: {cnt_tot_doi}, in PMID: {cnt_tot_pm}")
print(f"Skip PXD dataset doi: {skip_PXD_dataset_doi}")

Total dx.doi.org found: 12047
Total pubmed found: 19987
Total matches of gse accession codes: in DOI: 1498, in PMID: 3410
Skip PXD dataset doi: 2840


In [23]:
with open(output_ground_truth_file, 'w') as f:
    json.dump(input_ground_truth, f, indent=4) # Save to JSON

In [24]:
len(input_ground_truth)

34879